<a href="https://colab.research.google.com/github/yunsing/Compsci361/blob/master/fpmining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Frequent Pattern Mining

Two phase algorithm:
(1) the algorithm counts occurrence of items (attribute-value pairs) in the dataset, and stores them to 'header table'. 
(2) it builds the FP-tree structure by inserting instances. Items in each instance have to be sorted by descending order of their frequency in the dataset, so that the tree can be processed quickly. Items in each instance that do not meet minimum coverage threshold are discarded.



Orange add-on for enumerating frequent itemsets and association rules mining.

Documentation: http://orange3-associate.readthedocs.org/


In [0]:
!pip install -U Orange3-Associate
!pip install -U Orange3

We can try it with a larger and more real-world database of categorical values.

The zoo dataset is from UCI repository. https://archive.ics.uci.edu/ml/datasets/zoo

In [0]:
from orangecontrib.associate.fpgrowth import *

import Orange
data = Orange.data.Table('zoo')
data

We need to transform the data into a usable form for Orange3, we  perform a one-hot transformation.

In [0]:
X, mapping = OneHot.encode(data, include_class=True)

We want itemsets with minimum support of 40% support.

In [0]:
itemsets = dict(frequent_itemsets(X, .4))
len(itemsets)

The transaction-coded items corresponding to class values are.

In [0]:
class_items = {item
...                for item, var, _ in OneHot.decode(mapping, data, mapping)
...                if var is data.domain.class_var}

sorted(class_items)

That makes sense as our class variable has seven values.

In [0]:
data.domain.class_var.values

Now we can generate all association rules that have consequent equal to one of the class values and rules have a minimum confidence of 80% confidence.

In [0]:
rules = [(P, Q, supp, conf)
...          for P, Q, supp, conf in association_rules(itemsets, .8)
...          if len(Q) == 1 and Q & class_items]
len(rules)

In [0]:
rules

To make them more helpful, we can use mapping to transform the rules’ items back into table domain values, e.g. for first ten rules.

In [0]:
names = {item: '{}={}'.format(var.name, val)
...          for item, var, val in OneHot.decode(mapping, data, mapping)}
for ante, cons, supp, conf in rules[:10]:
...     print(', '.join(names[i] for i in ante), '-->',
...           names[next(iter(cons))],
...           '(supp: {}, conf: {})'.format(supp, conf))

**Question 1:**
What happens when you change the minmum support value? Try increasing and decreasing it? Are the results what you expected?  You may use the snippet of code below to help answer these questions.

**Question 2: **
What happens when you change the minmum support value? Try increasing and decreasing it? Are the results what you expected?


**Question 3: **
In association rules normally the consequent does not require you to have a specified class value. In the example above we have constrained the problems such that the consequent is of size 1 with a specific class. If you remove this constraint what happens?

In [0]:
itemsetLength = list()
minsupLength = list()
for i in range (1, 10):
  minsup = i*0.10
  
  itemsets = dict(frequent_itemsets(X, minsup))
  itemsetLength.append(len(itemsets))
  minsupLength.append(minsup)

  
import matplotlib.pyplot as plt
plt.plot(minsupLength, itemsetLength, 'ro')
plt.ylabel('number of itemset')
plt.xlabel('minsup')
plt.show()
